# SharePoint File Manager Notebook

This Snowflake notebook provides a user interface for managing SharePoint files directly from Snowflake using Streamlit.

## Features

* Connect to SharePoint using site URL and credentials
* Browse files in the default document library
* Download selected files to local storage
* Secure access through Snowflakes network rules and external access integration

## Prerequisites

* Access to a Snowflake account with appropriate permissions
* SharePoint site URL and valid credentials
* Required Python packages:
  - streamlit
  - office365-runtime
  - office365-sharepoint

## Setup

The notebook consists of two main components:

1. **Network Configuration (SQL)**
   * Creates network rules for SharePoint domains
   * Sets up external access integration for secure communication

2. **User Interface (Python)**
   * Provides a Streamlit-based interface for:
     - SharePoint authentication
     - File browsing
     - File downloading

## Usage

1. Enter your SharePoint site URL
2. Provide your username (email)
3. Enter your password
4. Click "Connect" to authenticate
5. Select a file from the displayed list
6. Click "Download Selected File" to save the file locally

## Security Notes

* Credentials are handled securely through Streamlits password input
* Network access is restricted to SharePoint domains
* External access integration ensures secure communication


In [ ]:
/* Create network rule for SharePoint domains */
CREATE
OR REPLACE NETWORK RULE sharepoint_network_rule MODE = EGRESS TYPE = HOST_PORT VALUE_LIST = ('*.sharepoint.com', '*.microsoftonline.com');
/* Create external access integration */
CREATE
OR REPLACE EXTERNAL ACCESS INTEGRATION sharepoint_integration ALLOWED_NETWORK_RULES = (sharepoint_network_rule) ENABLED = true;

GRANT USAGE ON INTEGRATION sharepoint_integration TO ROLE public;

In [ ]:
import streamlit as st
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
import os

# Set up the Streamlit interface
st.title("SharePoint File Manager")

# Input fields for SharePoint connection
with st.form("sharepoint_connection"):
    site_url = st.text_input("SharePoint Site URL")
    username = st.text_input("Username (email)")
    password = st.text_input("Password", type="password")
    submit_button = st.form_submit_button("Connect")

if submit_button and site_url and username and password:
    try:
        # Create SharePoint context
        ctx = ClientContext(site_url).with_credentials(
            UserCredential(username, password)
        )
        
        # Get the root folder
        root_folder = ctx.web.default_document_library().root_folder
        ctx.load(root_folder)
        ctx.execute_query()
        
        # Get all files in the document library
        files = root_folder.files
        ctx.load(files)
        ctx.execute_query()
        
        # Display files in a table
        if files:
            file_data = []
            for file in files:
                file_data.append({
                    "Name": file.properties["Name"],
                    "Modified": file.properties["TimeLastModified"],
                    "Size (KB)": round(file.properties["Length"] / 1024, 2)
                })
            
            st.write("### Available Files")
            selected_file = st.selectbox(
                "Select a file to download",
                options=[f["Name"] for f in file_data]
            )
            
            if st.button("Download Selected File"):
                for file in files:
                    if file.properties["Name"] == selected_file:
                        # Create download folder if it doesn't exist
                        if not os.path.exists("downloads"):
                            os.makedirs("downloads")
                        
                        # Download the file
                        with open(f"downloads/{selected_file}", "wb") as local_file:
                            file.download(local_file).execute_query()
                        st.success(f"File downloaded successfully to downloads/{selected_file}")
                        break
        else:
            st.info("No files found in the document library")
            
    except Exception as e:
        st.error(f"Error connecting to SharePoint: {str(e)}")
